In [1]:
from random import randint
from xml import dom
import boto3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import csv
from bs4 import BeautifulSoup 
from credentials import *

In [ ]:
# from selenium inport dev tools
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


In [10]:
def init_driver():
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    main_driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    main_driver.maximize_window()
    return main_driver

In [11]:
main_driver = init_driver()

In [62]:
def scrape_emails_from_google(main_driver,url):
    # main_driver.get(url)
    emails=[]
    source=main_driver.page_source
    soup=BeautifulSoup(source,'html.parser')    

    email_div=soup.find('div',class_='dURPMd').find_all('div',class_='MjjYud')

    for email in email_div:
        company=email.find('h3','LC20lb MBeuO DKV0Md').text
        email=get_email(email)
        emails.append({'company':company,'email':email})
    return emails

In [81]:
def get_email(email_div_temp):
    temp=email_div_temp.find('div',class_='VwiC3b yXK7lf lVm3ye r025kc hJNv6b Hdw6tb').text.split(' ')
    for i in temp:
        if '@' in i:
            if i[-1] == '.':
                return i[:-1]
            return i
    return None


In [56]:
gsearch_url="https://www.google.com/search?q=site:facebook.com+%22Roofing%22+AND+London+AND+Gmail&sca_esv=369bb2d90aa2509e&sca_upv=1&cs=0&tbas=0&sxsrf=ACQVn08h49W1RtrSVTzziYYOCnAjCt5A8Q:1713766575143&source=lnt&tbs=qdr:m&sa=X&ved=2ahUKEwifwMH_ldWFAxXthq8BHR7wB1sQpwV6BAgFEAo&biw=1821&bih=844&dpr=0.75"


In [82]:
Emails_data=scrape_emails_from_google(main_driver,gsearch_url)

In [83]:
with open('emails.csv', mode='w') as file:
    writer = csv.writer(file)
    writer.writerow(['Company', 'Email'])
    for email in Emails_data:
        writer.writerow([email['company'], email['email']])
print('done')


done


In [71]:
main_driver.get(gsearch_url)

In [72]:
source=main_driver.page_source
soup=BeautifulSoup(source,'html.parser')    

email_div=soup.find('div',class_='dURPMd').find_all('div',class_='MjjYud')

In [2]:
import json
import requests

In [5]:
response=requests.get('https://www.yellowpages.com/atlanta-ga/mip/southern-comfort-movers-523090613?lid=1002177179758')
json_obj_123 = json.loads(BeautifulSoup(response.text).find_all("script", type="application/ld+json")[0].text)

In [6]:
print(json_obj_123)

{'@context': 'http://schema.org', '@type': 'http://schema.org/MovingCompany', '@id': '/atlanta-ga/mip/southern-comfort-movers-523090613', 'name': 'Southern Comfort Movers', 'address': {'@type': 'PostalAddress', 'addressCountry': 'US', 'streetAddress': '4780 Ashford Dunwoody Rd', 'addressLocality': 'Atlanta', 'addressRegion': 'GA', 'postalCode': '30338'}, 'geo': {'@type': 'GeoCoordinates', 'latitude': 33.93512, 'longitude': -84.33804}, 'telephone': '(404) 369-1227', 'openingHours': ['Mo-Sa 08:30-23:00', 'Su 09:30-21:00'], 'image': {'@type': 'ImageObject', 'url': 'https://i3.ypcdn.com/blob/9f9ed281edc7b18f68477d5ce712e943d648e5f6', 'contentUrl': 'https://i3.ypcdn.com/blob/9f9ed281edc7b18f68477d5ce712e943d648e5f6', 'thumbnailUrl': 'https://i3.ypcdn.com/blob/9f9ed281edc7b18f68477d5ce712e943d648e5f6_400x260_crop.jpg'}, 'review': [], 'url': 'https://southerncomfortmovers.com', 'paymentAccepted': 'check, cash, mastercard, visa', 'location': {'@type': 'Place', 'description': 'Ashford Place'}, 

In [8]:
test123 = {'company': 'Apple'}
test={
                'company': test123.get('company'),
            }

In [13]:
def extract_data_from_yellow_pages(main_driver,url,pages_start,pages_end):
    json_data=[]
    prospect_data=[]
    for i in range(pages_start, pages_end+1):
        if i == 1:
            main_driver.get(url)
        else:
            if '?' in url:
                main_driver.get(f"{url}&page={i}")
            else:
                main_driver.get(f"{url}?page={i}")
        time.sleep(5)
        # the_soup = BeautifulSoup(response.text)
        the_soup = BeautifulSoup(main_driver.page_source)
        json_tag = the_soup.find_all("script", type="application/ld+json")
        json_obj = json.loads(json_tag[1].text)
        for item in json_obj:
            time.sleep(1)
            response_ = requests.get(item['url'])
            the_soup_123 = BeautifulSoup(response_.text)
            json_tag_123 = the_soup_123.find_all("script", type="application/ld+json")
            json_obj_123 = json.loads(json_tag_123[0].text)
            if json_obj_123:
                json_data.append(json_obj_123)
                print(json_obj_123)
                if 'email' in json_obj_123:
                    address_object=json_obj_123.get('address')
                    if address_object:
                        address= address_object.get('streetAddress'),
                        city= address_object.get('addressLocality'),
                        state= address_object.get('addressRegion'),
                        country= address_object.get('addressCountry')
                    else:
                        address= None
                        city= None
                        state= None
                        country= None

                    prospect_data.append({
                        'company': json_obj_123.get('name'),
                        'email': json_obj_123.get('email').replace('mailto:', ''),
                        'phone': json_obj_123.get('telephone'),
                        'website': json_obj_123.get('url'),
                        'address': address,
                        'city': city,
                        'state': state,
                        'country': country
                        
                    })

    return prospect_data,json_data

    



In [26]:
main_driver=init_driver()

In [39]:
response_ = requests.get('https://www.yellowpages.com/las-vegas-nv/mip/golden-gaming-14033721')
print(response_.text)

<!DOCTYPE html><html lang="en"><head><script async src="https://www.googletagmanager.com/gtag/js?id=G-0EQTJQH34W"></script><script>window.dataLayer = window.dataLayer || [];

function gtag(){dataLayer.push(arguments);}

gtag('js', new Date());
gtag('config', 'G-0EQTJQH34W');

if (document.cookie.indexOf('optOut') !== -1) {
    gtag('set', 'allow_google_signals', 'false');
    gtag('set', 'ad_storage', 'denied');
    gtag('set', 'analytics_storage', 'denied');
    gtag('set', 'ads_data_redaction', 'true');
    gtag('set', 'allow_ad_personalization_signals', 'false');
    gtag('set', 'restricted_data_processing', 'true');
}</script><meta name="charset" content="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1, user-scalable=no"><title>Golden Gaming - Las Vegas, NV 89121</title><meta name="description" content="Get reviews, hours, directions, coupons and more for Golden Gaming. Search for othe

In [15]:
url="https://www.yellowpages.com/search?search_terms=divorce-attorneys&geo_location_terms=fl"
pages=100
main_data,json_data=extract_data_from_yellow_pages(main_driver,url,31,50)

{'@context': 'http://schema.org', '@type': 'http://schema.org/LegalService', '@id': '/gainesville-fl/mip/barbarette-quirk-451178234', 'name': 'Barbarette & Quirk', 'address': {'@type': 'PostalAddress', 'addressCountry': 'US', 'streetAddress': '5538-A NW 43rd Street', 'addressLocality': 'Gainesville', 'addressRegion': 'FL', 'postalCode': '32653'}, 'geo': {'@type': 'GeoCoordinates', 'latitude': 29.70529, 'longitude': -82.38993}, 'telephone': '(352) 356-4999', 'openingHours': ['Mo-Fr 09:00-17:00'], 'image': {'@type': 'ImageObject', 'url': 'https://i4.ypcdn.com/blob/5262db28b4e743942ff30c35dcc25eb180d4d24a', 'contentUrl': 'https://i4.ypcdn.com/blob/5262db28b4e743942ff30c35dcc25eb180d4d24a', 'thumbnailUrl': 'https://i4.ypcdn.com/blob/5262db28b4e743942ff30c35dcc25eb180d4d24a_400x260_crop.jpg'}, 'url': 'https://www.bqlawoffice.com', 'paymentAccepted': 'mastercard, all major credit cards, amex, diners club, discover, visa'}
{'@context': 'http://schema.org', '@type': 'http://schema.org/LegalSer

In [43]:
import os

In [46]:


with open(os.getcwd()+'/casino-nv-us.csv', mode='w',newline='') as file:
        writer = csv.DictWriter(file, fieldnames=main_data[0].keys())
        writer.writeheader()
        for row in main_data:
            writer.writerow(row)

In [71]:
page_sources[250]

'https://www.yellowpages.com/atlanta-ga/mip/moving-squad-456496215'

In [67]:
data[-1]

{'@context': 'http://schema.org',
 '@type': 'http://schema.org/LocalBusiness',
 '@id': '/atlanta-ga/mip/moving-squad-456496215',
 'name': 'Moving Squad',
 'address': {'@type': 'PostalAddress',
  'addressCountry': 'US',
  'streetAddress': '72 Walton St NW',
  'addressLocality': 'Atlanta',
  'addressRegion': 'GA',
  'postalCode': '30303'},
 'geo': {'@type': 'GeoCoordinates',
  'latitude': 33.756496,
  'longitude': -84.39108},
 'telephone': '(404) 891-6498',
 'openingHours': ['Mo-Fr 09:00-20:00', 'Sa 10:00-15:00'],
 'image': {'@type': 'ImageObject',
  'url': 'https://i1.ypcdn.com/ypu/images/default-thumbnails/thumbnail-1.svg?addecef'},
 'review': [],
 'paymentAccepted': 'visa, master card, amex, discover, debit, check',
 'email': 'mailto:info@movingcost.com'}

In [72]:
for url in page_sources[251:]:
    response = requests.get(url)
    the_soup_123 = BeautifulSoup(response.text)
    json_tag_123 = the_soup_123.find_all("script", type="application/ld+json")
    json_obj_123 = json.loads(json_tag_123[0].text)
    data.append(json_obj_123)
    print(json_obj_123)

{'@context': 'http://schema.org', '@type': 'http://schema.org/LocalBusiness', '@id': '/atlanta-ga/mip/apartment-movers-atlanta-9211', 'name': 'Apartment Movers Atlanta', 'address': {'@type': 'PostalAddress', 'addressCountry': 'US', 'streetAddress': '303 Parkway Dr NE', 'addressLocality': 'Atlanta', 'addressRegion': 'GA', 'postalCode': '30312'}, 'geo': {'@type': 'GeoCoordinates', 'latitude': 33.76298, 'longitude': -84.37393}, 'telephone': '(404) 671-9537', 'openingHours': ['Mo-Su 09:00-17:00'], 'image': {'@type': 'ImageObject', 'url': 'https://i1.ypcdn.com/ypu/images/default-thumbnails/thumbnail-1.svg?addecef'}, 'review': [], 'url': 'http://www.apartmentmovers1.us', 'paymentAccepted': 'visa, master card, amex, discover, atm, check, diners club'}
{'@context': 'http://schema.org', '@type': 'http://schema.org/LocalBusiness', '@id': '/atlanta-ga/mip/moving-help-472234858', 'name': 'Moving Help', 'address': {'@type': 'PostalAddress', 'addressCountry': 'US', 'streetAddress': '816 Pine Shoals 

KeyboardInterrupt: 

In [58]:
data[0]['address']['addressCountry']

'US'

In [65]:
prospect_data=[]
for item in data:
    if 'email' in item:
        print(item)
        prospect_data.append(
            {
                'company': item['name'],
                'email': item['email'].replace('mailto:', ''),
            }
        )
        

{'@context': 'http://schema.org', '@type': 'http://schema.org/MovingCompany', '@id': '/atlanta-ga/mip/air-7-seas-transport-logistics-inc-457918084', 'name': 'AIR 7 SEAS Transport Logistics Inc', 'address': {'@type': 'PostalAddress', 'addressCountry': 'US', 'streetAddress': '560 Atlanta South Pkwy', 'addressLocality': 'Atlanta', 'addressRegion': 'GA', 'postalCode': '30349'}, 'aggregateRating': {'@type': 'AggregateRating', 'ratingValue': 5, 'reviewCount': 64}, 'geo': {'@type': 'GeoCoordinates', 'latitude': 33.611256, 'longitude': -84.40813}, 'telephone': '(888) 404-8124', 'openingHours': ['Mo-Fr 08:00-17:30', 'Sa 09:00-16:00', 'Su 10:00-15:00'], 'image': {'@type': 'ImageObject', 'url': 'https://i3.ypcdn.com/blob/db18eccc583f6a534dade2809c70cb895e06243a', 'contentUrl': 'https://i3.ypcdn.com/blob/db18eccc583f6a534dade2809c70cb895e06243a', 'thumbnailUrl': 'https://i3.ypcdn.com/blob/db18eccc583f6a534dade2809c70cb895e06243a_400x260_crop.jpg'}, 'review': [{'@type': 'Review', 'itemReviewed': 'A

In [43]:
the_soup = BeautifulSoup(response.text)

# find <script type="application/ld+json"> tag
json_tag = the_soup.find_all("script", type="application/ld+json")

# load to json onject
json_obj = json.loads(json_tag[1].text)
json_obj[0]

IndexError: list index out of range

In [1]:
# scraper for weedmaps

import os
import json
# list directory in sorted order
files = os.listdir('/Users/abhivora/Downloads/Cannabis_Data')
files.sort()



main_list=[]

for file_name in files:

    the_soup = BeautifulSoup(open(f"/Users/abhivora/Downloads/Cannabis_Data/{file_name}"))
    querries=the_soup.find_all('script', id='__NEXT_DATA__')

    # json
    data = json.loads(querries[0].text)
    listings_data=(data['props']['dehydratedState']['queries'][0]['state']['data']['data']['listings'])
    for listing in listings_data:
        main_list.append({'company': listing['name'],
                          'city': listing['city'],
                          'email': listing['email'],
                          'phone': listing['phoneNumber']})
        
# write main_list to csv
with open('cannabis_data.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['company', 'city', 'email', 'phone'])
    writer.writeheader()
    writer.writerows(main_list)
        

NameError: name 'files' is not defined

In [63]:
    

# find <ul data-testid="map-listings-list-wrapper" class="ListWrapper-sc-8b32aaa7-0 hKuVwl">
listings = the_soup.find('ul', {'data-testid': 'map-listings-list-wrapper'})
# find all <li> tags
listings = listings.find_all('li')

    

In [18]:

    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68


In [10]:
main_driver.get("https://weedmaps.com/dispensaries/in/united-states/district-of-columbia/washington")
the_soup = BeautifulSoup(main_driver.page_source, 'html.parser')

# find <ul data-testid="map-listings-list-wrapper" class="ListWrapper-sc-8b32aaa7-0 hKuVwl">
listings = the_soup.find('ul', {'data-testid': 'map-listings-list-wrapper'})
# find all <li> tags
listings = listings.find_all('li')

In [33]:
the_soup = BeautifulSoup(main_driver.page_source, 'html.parser')

print(the_soup)

<html lang="en"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="px-captcha" name="description"/>
<title>Access to this page has been denied</title>
<script src="/P750P64I/captcha/captcha.js?a=&amp;u=2bcdfcd0-fd71-11ee-a3ef-90c21b46ed45&amp;v=926ef34e-fd6f-11ee-878e-81c19c628ce7&amp;m=0"></script><style>@import url('https://fonts.googleapis.com/css2?family=Roboto:ital,wght@0,100;0,300;0,400;0,500;0,700;0,900;1,100;1,300;1,400;1,500;1,700;1,900&display=swap');  div.px-captcha-background { position: fixed; top: 0; left: 0; width: 100%; height: 100%; background-color: rgba(0, 0, 0, 0.31); } div.px-captcha-container { position: fixed; height: 340px; background-color: #fff; font-family: Roboto, sans-serif; } img.px-captcha-logo { display: block; margin: auto; padding: 53px 0 31px; } div.px-captcha-header { color: #626364;  font-size: 29px; font-weight: 500; margin: 67px 0 33px; line-height: 0.83; text-align: center; } div.px

In [20]:
listings[0]

<li class="ListItem-sc-8b32aaa7-1 gPRBLE"><div data-testid="map-listings-list-card-wrapper"><div class="_StyledDiv-sc-dcf1723c-0 iDOLjq" data-test-id="card"><div class="Card-sc-fd9387d6-1 kvnyNa"><div class="src__Box-sc-1sbtrzs-0 src__Flex-sc-1sbtrzs-1 Main-sc-5f639846-6 hNEOfd eqdOIn fVGrob" data-testid="map-listing-card"><div class="src__Box-sc-1sbtrzs-0 src__Flex-sc-1sbtrzs-1 AvatarContainer-sc-5f639846-14 hNEOfd eqdOIn gRqYMG"><a aria-label="Takoma Wellness Center" href="/dispensaries/takoma-wellness-center-2-2"><div><div class="AvatarStyled-sc-5e11720b-0 gkBHjm" data-testid="avatar-container" shape="square" size="96"><div class="LazyImgWrapper-sc-ac9b2a78-0 bjuNvy" data-testid="lazy-img-wrapper"><noscript data-testid="noscript"><img alt="" data-testid="lazy-avatar-img" height="96" loading="lazy" src="https://weedmaps.com/images/dispensaries/000/036/837/avatar/1698606585-takoma-logonew.jpg?auto=format&amp;w=96&amp;h=96" srcset="https://weedmaps.com/images/dispensaries/000/036/837/a

In [17]:
listings[0].find('a').get('aria-label')

'Takoma Wellness Center'

In [25]:
print(listings.find('email'))

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [18]:
listings[0].find('a').get('href')

'/dispensaries/takoma-wellness-center-2-2'

In [19]:
print('https://weedmaps.com/'+listings[0].find('a').get('href'))

https://weedmaps.com//dispensaries/takoma-wellness-center-2-2


In [10]:
def login_linkedin_sales_navigator(main_driver,user):

    main_driver.get("https://www.linkedin.com/login?session_redirect=%2Fsales&_f=navigator&fromSignIn=true/1000")
    username,password = get_sales_navigator_creds(user)
    time.sleep(5)

    main_driver.find_element(By.XPATH, "/html/body/div/main/div[2]/form/div[1]/input").send_keys(username)
    main_driver.find_element(By.XPATH, "/html/body/div/main/div[2]/form/div[2]/input").send_keys(password)

    time.sleep(3)

    main_driver.find_element(By.XPATH, "/html/body/div/main/div[2]/form/div[3]/button").click()


In [11]:
url="https://www.linkedin.com/sales/search/company?query=(spellCorrectionEnabled%3Atrue%2Cfilters%3AList((type%3ACOMPANY_HEADCOUNT%2Cvalues%3AList((id%3AB%2Ctext%3A1-10%2CselectionType%3AINCLUDED)))%2C(type%3AINDUSTRY%2Cvalues%3AList((id%3A99%2Ctext%3ADesign%2520Services%2CselectionType%3AINCLUDED)))%2C(type%3AREGION%2Cvalues%3AList((id%3A101165590%2Ctext%3AUnited%2520Kingdom%2CselectionType%3AINCLUDED))))%2Ckeywords%3ADesign%2520agency)&sessionId=CQMB7RqpS8avLMW50KU7xg%3D%3D"

In [12]:
main_driver.get(url)

In [23]:

main_driver.find_element(By.XPATH, "/html/body/main/div[1]/div[2]/div[1]/div[2]/div/button[1]").click()


In [ ]:
def automate_prospecting_sales_navigator(main_driver, url, list_name,num):
    main_driver.get(url)
    time.sleep(randint(5, 10))

    for i in range(num):

        #click on select all
        main_driver.find_element(By.XPATH, "/html/body/main/div[1]/div[2]/div[1]/div[2]/div/div[1]").click()
        time.sleep(randint(5, 10))

        # click on save button
        main_driver.find_element(By.XPATH, "/html/body/main/div[1]/div[2]/div[1]/div[2]/div/button[1]").click()
        time.sleep(randint(5, 10))

        #send list name
        main_driver.find_element(By.XPATH,'/html/body/div[7]/div[2]/div/div/div[2]/form/div[1]/div[2]/div/div/div/div[2]/div/div/div/div/div/div[1]/input').send_keys(list_name)
        time.sleep(randint(5, 10))

        #click on save button
        main_driver.find_element(By.XPATH,'/html/body/div[7]/div[2]/div/div/div[3]/button[1]').click()
        time.sleep(randint(5, 10))

        #refresh prospects
        main_driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[4]/div/div[2]/div/div/div/div[2]/div/div[2]/div[2]').click()
        time.sleep(randint(5, 10))


        # # next page
        # driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div/div[2]/div/div[2]/div/div/div/div/div/div[2]/div/div[4]/div/div/div/div/div[3]/div/div[2]/button[2]').click()
        # time.sleep(randint(5, 10))

In [8]:
main_driver.quit()

In [3]:
from methods import *

In [6]:
main_driver=init_driver()

In [10]:
login_apollo(main_driver=main_driver,user='achugh')


In [12]:
sort_by_name(main_driver=main_driver)

In [12]:
def extract_data(row,bucket_name,competitor_domain,competitor_name):
    com_linked_in=''
    linked_in=''
    name = row.findAll('td')[0].text.split(' ')[0]
    # error handling for linkedin
    try:
        linked_in = row.findAll('td')[0].select('a[href]')[0].get('href')
        if 'linkedin' not in linked_in:
            linked_in = row.findAll('td')[0].select('a[href]')[1].get('href')
            if 'linkedin' not in linked_in:
                linked_in = row.findAll('td')[0].select('a[href]')[2].get('href')
                if 'linkedin' not in linked_in:
                    linked_in = row.findAll('td')[0].select('a[href]')[3].get('href')
    except:
        pass    
    title = row.findAll('td')[1].text
    company = row.findAll('td')[2].text
    # error handling for company linkedin
    try:
        company_website = row.findAll('td')[2].select('a[href]')[1].get('href')
    except:
        company_website=""

    try:
        com_linked_in = row.findAll('td')[2].select('a[href]')[1].get('href')
        if 'linkedin' not in com_linked_in:
            com_linked_in = row.findAll('td')[2].select('a[href]')[2].get('href')
        else:
            pass
    except:
        pass
    email = row.findAll('td')[4].text
    industry = row.findAll('td')[6].text
    contact_loc = row.findAll('td')[8].text
    company_loc = row.findAll('td')[7].text.replace('N/A',contact_loc)
    
    company_domain = company_website.replace('https://','').replace('http://','').replace('/','')
    
    # print(name,linked_in,title,company,contact_loc,industry,email)
    return {'Name': name,
            'Email':email,
            'linked_in':linked_in,
            'company':company,
            'com_linked_in':com_linked_in,
            'title': title,
            'industry':industry,
            'contact_loc':contact_loc,
            'company_loc':company_loc,
            'company_domain':company_domain,
            }

In [13]:
def get_data(driver,page_count,competitor_domain,competitor_name,industry,country,date):
    industry = industry.lower().replace(' ','-')
    country = country.lower().replace(' ','-')
    date = date.replace('_','-').replace('/','-')
    csv_filename = f"{industry}-{country}-{date}.csv"
    bucket_name = f"{industry}-{country}-{date}"
    data = []
    for i in range(1,page_count+1):
        time.sleep(5)
        the_soup = BeautifulSoup(driver.page_source, 'html.parser')
        table=the_soup.findAll('tbody')
        for row in table:
            data.append(extract_data(row,bucket_name,competitor_domain,competitor_name))
        if i<page_count:
            driver.find_element(By.XPATH,"/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div[2]/div/div[4]/div/div/div/div/div[3]/div/div[2]/button[2]").click()
    return data,bucket_name,csv_filename

In [14]:
from random import randint


def automate1(driver, url, list_name,num):
    driver.get(url)
    time.sleep(randint(5, 10))

    for i in range(num):

        #click on select all
        driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div[2]/div/div[3]/div/div/div/div[1]/div[1]/div/button').click()
        time.sleep(randint(5, 10))

        #select all
        driver.find_element(By.XPATH,'/html/body/div[7]/div/div/div/div/div/a[1]').click()
        time.sleep(randint(5, 10))

        # #open add to list
        # driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div/div[2]/div/div[2]/div/div/div/div/div/div[2]/div/div[3]/div/div/div/div[1]/div[4]/button[3]').click()
        # time.sleep(randint(5, 10))

        # #click on add to list
        # driver.find_element(By.XPATH,'/html/body/div[7]/div/div/div/div/div/a').click()
        # time.sleep(randint(5, 10))

        # click on save button
        driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div[2]/div/div[3]/div/div/div/div[1]/div[3]/span[1]/button').click()
        time.sleep(randint(5, 10))

        #send list name
        driver.find_element(By.XPATH,'/html/body/div[7]/div[2]/div/div/div[2]/form/div[1]/div[2]/div/div/div/div[2]/div/div/div/div/div/div[1]/input').send_keys(list_name)
        time.sleep(randint(5, 10))

        #click on save button
        driver.find_element(By.XPATH,'/html/body/div[7]/div[2]/div/div/div[3]/button[1]').click()
        time.sleep(randint(5, 10))

        #refresh prospects
        driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[4]/div/div[2]/div/div/div/div[2]/div/div[2]/div[2]').click()
        time.sleep(randint(5, 10))


        # # next page
        # driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div/div[2]/div/div[2]/div/div/div/div/div/div[2]/div/div[4]/div/div/div/div/div[3]/div/div[2]/button[2]').click()
        # time.sleep(randint(5, 10))

In [14]:
def extract_data(row,bucket_name,competitor_domain,competitor_name):
    com_linked_in=''
    linked_in=''
    name = row.findAll('td')[0].text.split(' ')[0]
    # error handling for linkedin
    try:
        linked_in = row.findAll('td')[0].select('a[href]')[0].get('href')
        if 'linkedin' not in linked_in:
            linked_in = row.findAll('td')[0].select('a[href]')[1].get('href')
            if 'linkedin' not in linked_in:
                linked_in = row.findAll('td')[0].select('a[href]')[2].get('href')
                if 'linkedin' not in linked_in:
                    linked_in = row.findAll('td')[0].select('a[href]')[3].get('href')
    except:
        pass    
    title = row.findAll('td')[1].text
    company = row.findAll('td')[2].text
    # error handling for company linkedin
    try:    
        company_website = row.findAll('td')[2].select('a[href]')[1].get('href')
    except:
        company_website=""

    try:
        com_linked_in = row.findAll('td')[2].select('a[href]')[1].get('href')
        if 'linkedin' not in com_linked_in:
            com_linked_in = row.findAll('td')[2].select('a[href]')[2].get('href')
            if 'linkedin' not in com_linked_in:
                com_linked_in = row.findAll('td')[2].select('a[href]')[3].get('href')
                if 'linkedin' not in com_linked_in:
                    com_linked_in = row.findAll('td')[2].select('a[href]')[4].get('href')
    except:
        pass
    email = row.findAll('td')[4].text
    industry = row.findAll('td')[6].text
    contact_loc = row.findAll('td')[8].text
    company_loc = row.findAll('td')[7].text.replace('N/A',contact_loc)
    
    company_domain = company_website.replace('https://','').replace('http://','').replace('/','')
    
    # print(name,linked_in,title,company,contact_loc,industry,email)
    return {'Name': name,
            'Email':email,
            'linked_in':linked_in,
            'company':company,
            'com_linked_in':com_linked_in,
            'title': title,
            'industry':industry,
            'contact_loc':contact_loc,
            'company_loc':company_loc,
            'company_domain':company_domain
            }

In [4]:
main_driver=init_driver()

In [7]:
login_apollo(main_driver=main_driver,user='tools')

In [13]:
url='https://app.apollo.io/#/people?finderViewId=5b6dfc5a73f47568b2e5f11c&personTitles[]=owner&personTitles[]=Founder&personTitles[]=founder%20and%20ceo&personTitles[]=CEO&personTitles[]=co%20founder&personTitles[]=CMO&personTitles[]=managing%20director&page=1&organizationNumEmployeesRanges[]=1%2C10&contactEmailStatusV2[]=likely_to_engage&contactEmailStatusV2[]=verified&qOrganizationKeywordTags[]=Yoga&qOrganizationKeywordTags[]=yoga&qOrganizationKeywordTags[]=yoga%20classes&qOrganizationKeywordTags[]=yoga%20center&qOrganizationKeywordTags[]=yoga%20teacher&qOrganizationKeywordTags[]=meditation&qOrganizationKeywordTags[]=spiritual%20practice&qOrganizationKeywordTags[]=sustainable%20living&qOrganizationKeywordTags[]=retreat%20center&qOrganizationKeywordTags[]=hindu%20center&qOrganizationKeywordTags[]=ashram&includedOrganizationKeywordFields[]=tags&includedOrganizationKeywordFields[]=name&organizationLocations[]=United%20States&prospectedByCurrentTeam[]=no&sortByField=person_name.raw&sortAscending=true'
automate_prospecting_apollo(main_driver,url,'Yoga USA 1',20)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[7]/div/div/div/div/div/a[1]"}
  (Session info: chrome=124.0.6367.62); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100e7e8dc chromedriver + 4368604
1   chromedriver                        0x0000000100e76d70 chromedriver + 4337008
2   chromedriver                        0x0000000100a9ac04 chromedriver + 289796
3   chromedriver                        0x0000000100adce00 chromedriver + 560640
4   chromedriver                        0x0000000100b155ec chromedriver + 792044
5   chromedriver                        0x0000000100ad1ab4 chromedriver + 514740
6   chromedriver                        0x0000000100ad250c chromedriver + 517388
7   chromedriver                        0x0000000100e42df8 chromedriver + 4124152
8

In [ ]:
login

In [15]:
link="https://app.apollo.io/#/people?finderViewId=5b6dfc5a73f47568b2e5f11c&personTitles[]=owner&personTitles[]=Founder&personTitles[]=founder%20and%20ceo&personTitles[]=CEO&personTitles[]=co%20founder&personTitles[]=CMO&personTitles[]=managing%20director&page=1&organizationNumEmployeesRanges[]=1%2C10&contactEmailStatusV2[]=likely_to_engage&contactEmailStatusV2[]=verified&qOrganizationKeywordTags[]=Yoga&qOrganizationKeywordTags[]=yoga&qOrganizationKeywordTags[]=yoga%20classes&qOrganizationKeywordTags[]=yoga%20center&qOrganizationKeywordTags[]=yoga%20teacher&qOrganizationKeywordTags[]=meditation&qOrganizationKeywordTags[]=spiritual%20practice&qOrganizationKeywordTags[]=sustainable%20living&qOrganizationKeywordTags[]=retreat%20center&qOrganizationKeywordTags[]=hindu%20center&qOrganizationKeywordTags[]=ashram&includedOrganizationKeywordFields[]=tags&includedOrganizationKeywordFields[]=name&organizationLocations[]=United%20States&prospectedByCurrentTeam[]=no&sortByField=person_name.raw&sortAscending=true"
list_name="Yoga USA 1"
automate1(main_driver,link, list_name,20)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[7]/div/div/div/div/div/a[1]"}
  (Session info: chrome=124.0.6367.62); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100e7e8dc chromedriver + 4368604
1   chromedriver                        0x0000000100e76d70 chromedriver + 4337008
2   chromedriver                        0x0000000100a9ac04 chromedriver + 289796
3   chromedriver                        0x0000000100adce00 chromedriver + 560640
4   chromedriver                        0x0000000100b155ec chromedriver + 792044
5   chromedriver                        0x0000000100ad1ab4 chromedriver + 514740
6   chromedriver                        0x0000000100ad250c chromedriver + 517388
7   chromedriver                        0x0000000100e42df8 chromedriver + 4124152
8

In [18]:
main_driver.quit()

In [19]:
main_driver=init_driver()
login_apollo(main_driver=main_driver,user='tools')

In [20]:
# pages=get_page_count(driver=main_driver)
pages=20
competitor_name=""
competitor_domain=""
industry="Yoga"
country="us"
date="01-05-2024"


In [21]:
main_data,bucket_name,csv_file_name=get_data(main_driver=main_driver,page_count=pages,industry=industry,country=country,date=date)

In [5]:
create_s3_bucket(s3=init_s3(),bucket_name="real-estate-vermont-us-29-02-24")

True

In [22]:
csv_url=write_to_csv(data=main_data,filename=csv_file_name,bucket_name=bucket_name)

In [23]:
print(csv_url)

https://yoga-us-01-05-2024.s3.amazonaws.com/yoga-us-01-05-2024.csv


In [7]:
login_semrush(main_driver)

In [154]:
screenshot_to_s3(s3=init_s3(),driver=main_driver,bucket_name=bucket_name,domain=competitor_domain,region=country)

True

In [39]:
s3=init_s3()

In [14]:
update_screenshots_to_s3(s3=init_s3(),driver=main_driver,bucket_name=bucket_name,data=main_data,region=country)

InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x000000010480a814 chromedriver + 4040724
1   chromedriver                        0x0000000104802a18 chromedriver + 4008472
2   chromedriver                        0x00000001044756a8 chromedriver + 284328
3   chromedriver                        0x00000001044acc14 chromedriver + 510996
4   chromedriver                        0x00000001044d71c0 chromedriver + 684480
5   chromedriver                        0x00000001044d2720 chromedriver + 665376
6   chromedriver                        0x00000001044d209c chromedriver + 663708
7   chromedriver                        0x0000000104446d74 chromedriver + 93556
8   chromedriver                        0x00000001047cfa3c chromedriver + 3799612
9   chromedriver                        0x00000001047d3f1c chromedriver + 3817244
10  chromedriver                        0x00000001047b8298 chromedriver + 3703448
11  chromedriver                      

In [185]:


main_driver.quit()

In [8]:
screenshot_to_s3(s3=init_s3(),driver=main_driver,bucket_name="real-estate-ny-us-19-02-2024",domain="www.peterashe.com",region="us")

True

In [133]:
data = update_competitor_details(data=main_data[1000:],competitor_name="Eagle Rock Properties",competitor_domain="www.eaglerockproperties.com",bucket_name="real-estate-ny-us-20-02-2024")


In [146]:
data=update_competitor_and_company_details(data=main_data[1000:],competitor_name="Eagle Rock Properties",competitor_domain="www.eaglerockproperties.com",bucket_name="real-estate-ny-us-20-02-2024")

In [151]:
write_to_csv(data=data,filename=csv_file_name,bucket_name=bucket_name)

'https://real-estate-ny-us-20-02-2024.s3.amazonaws.com/real-estate-ny-us-20-02-2024.csv'

In [1]:
# read csv into dictionary
from pandas import read_csv
df = read_csv('real-estate-ny-us-19-02-2024.csv')
data = df.to_dict(orient='records')

In [30]:
lookup_data=read_csv("clicked.csv").to_dict(orient='records')


In [25]:
email=str(data[4]["Email"]).lower().strip()
email1=str(lookup_data[0]["Email"]).lower().strip()

email==email1

True

In [31]:
linkedin_url=[]
for item in lookup_data:
    email=str(item["Email"]).lower().strip()
    for data_item in data:
        email1=str(data_item["Email"]).lower().strip()
        if email==email1:
            print(email)
            print(data_item["linked_in"])
            print("found")
            linkedin_url.append(data_item["linked_in"])
            break
    else:
        print("not found")
        print(email)
        print(email1)
        print("not found")
        break

akleiman@nexusci.com
http://www.linkedin.com/in/aaron-kleiman-bb2502122
found
abickoff@sterlingprojectdevelopment.com
http://www.linkedin.com/in/adam-bickoff-4309548
found
a.albert@tmdre.com
http://www.linkedin.com/in/adrienne-albert-59798413
found
alec@olcny.com
http://www.linkedin.com/in/alec-ornstein-2530666
found
asvetlakou@ripcony.com
http://www.linkedin.com/in/alex-svetlakou-a90a86161
found
acaiola@bandlmanagement.com
http://www.linkedin.com/in/alfred-caiola-35112419
found
alyssa.thomas@rsequity.com
http://www.linkedin.com/in/alyssa-thomas-43b69723
found
awetzler@sterlingprojectdevelopment.com
http://www.linkedin.com/in/andrewwetzler
found
carol@velocitytitle.com
#/contacts/65d2eac4db6b25000166a096
found
carol@premierequities.com
http://www.linkedin.com/in/carol-hensley-chester-8749bb7
found
charlotte.banghart@rsequity.com
http://www.linkedin.com/in/charlotte-banghart-9b18027b
found
chris.murray@redstoneequity.com
http://www.linkedin.com/in/chris-murray-9642a09
found
christina@ve

In [34]:
main_driver.get("https://www.linkedin.com/in/")

In [84]:



for i in range(31,40):
    main_driver.get(linkedin_url[i])
    time.sleep(10)

KeyboardInterrupt: 

In [80]:
create_s3_bucket(s3=init_s3(),bucket_name="test-bucket-versioning-hotbit")

AttributeError: 'S3' object has no attribute 'BucketVersioning'

In [ ]:
import imaplib
import email
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Function to connect to Gmail
def connect_to_gmail(username, password):
    mail = imaplib.IMAP4_SSL('imap.gmail.com')
    mail.login(username, password)
    return mail

# Function to search for email thread
def search_email(mail, search_query):
    mail.select('inbox')
    result, data = mail.search(None, '(SUBJECT "{}")'.format(search_query))
    return data[0].split()[-1]  # Assuming the latest email in the thread is what we want

# Function to fetch email
def fetch_email(mail, email_id):
    result, data = mail.fetch(email_id, '(RFC822)')
    raw_email = data[0][1]
    return email.message_from_bytes(raw_email)

# Function to compose email
def compose_email(to, subject, body, references=None):
    msg = MIMEMultipart()
    msg['From'] = "your_email@gmail.com"
    msg['To'] = to
    msg['Subject'] = subject
    
    # If there are references, add them to the email headers
    if references:
        msg['References'] = references
        msg['In-Reply-To'] = references

    # Create a MIMEText object with HTML content
    print(type(body))
    print(body)
    if isinstance(body, bytes):
        body = body.decode('utf-8')
    html_body = MIMEText(body, 'html')
    msg.attach(html_body)
    
    return msg

# Function to save email as draft
def save_as_draft(mail, draft):
    mail.append('INBOX', None, None, draft)

# Function to draft initial email
def draft_initial_email(mail, to, subject, body):
    draft = compose_email(to, subject, body)
    save_as_draft(mail, draft)

# Function to draft trailing email
def draft_trailing_email(mail, to, subject, body, references):
    draft = compose_email(to, subject, body, references)
    save_as_draft(mail, draft)


def draft_email_imap(data, initial, follow_up):
    # Example usage
    username = 'avora@hotbitdigital.com'
    password = 'unsw vvjr nzbt yhwd'

    mail = connect_to_gmail(username, password)
    if initial:
    # Draft initial email
        to = data['receiver']
        initial_subject = f"Discover How {data['company']} Can Soar in Vermont!"
        initial_body = '<html><body><p>Dear recipient,</p><p>This is the initial email...</p></body></html>'
        draft_initial_email(mail, to, initial_subject, initial_body)

    if follow_up:
        # Draft trailing emails with different subject lines in the same thread
        initial_subject = f"Discover How {data['company']} Can Soar in Vermont!"
        initial_references = search_email(mail, initial_subject)
        trailing_subjects = [f"Quick Check-In: Enhancing {data['company']}'s Visibility", 'Follow-up 2 Subject', 'Follow-up 3 Subject', 'Follow-up 4 Subject', 'Follow-up 5 Subject']


        draft_trailing_email(mail, data['receiver'], trailing_subjects[follow_up-1], get_trailing_body(data,follow_up=follow_up), initial_references)
